In [ ]:
!pip install Bio
!pip install import-ipynb
# !pip install wandb

     |████████████████████████████████| 269 kB 5.4 MB/s 
     |████████████████████████████████| 2.3 MB 43.3 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
# import wandb
from datetime import datetime

# so we can import utils notebook (delete if working on Pycharm), you might need to change it to your working directory path
%cd "/content/drive/MyDrive/ColabNotebooks"
import import_ipynb
import utils

/content/drive/.shortcut-targets-by-id/13JKG3wgYlOO5t5AoOpkix7QiZamDhSEp/ColabNotebooks
importing Jupyter notebook from utils.ipynb
Num GPUs Available:  1


In [ ]:
###############################################################################
#                                                                             #
#              Parameters you can change, but don't have to                   #
#                                                                             #
###############################################################################


# number of ResNet blocks for the first ResNet and the kernel size.
RESNET_1_BLOCKS = 2
RESNET_1_KERNEL_SIZE = 3
RESNET_1_KERNEL_NUM = 43

###############################################################################
#                                                                             #
#                        Parameters you need to choose                        #
#                                                                             #
###############################################################################


# number of ResNet blocks for the second ResNet, dilation list to repeat and the kernel size.

RESNET_2_BLOCKS = 5
RESNET_2_KERNEL_SIZE = 7  # good start may be 3/5
RESNET_2_KERNEL_NUM = 53 # DO NOT MAKE IT 1!
DILATION = [1]
WANTED_M = len(DILATION)  # len of DILATION to be randomize by 'wandb' tool

# percentage of dropout for the dropout layer
DROPOUT = 0.29715545068   # good start may be 0.1-0.5

# number of epochs, Learning rate and Batch size
EPOCHS = 10
LR = 0.0051378873577  # good start may be 0.0001/0.001/0.01
BATCH = 128  # good start may be 32/64/128


def get_time():
    now = datetime.now()
    return now.strftime("%d-%m-%Y__%H-%M-%S")


def resnet_block(input_layer, kernel_size, kernel_num, dialation=1):
    bn1 = layers.BatchNormalization()(input_layer)
    conv1d_layer1 = layers.Conv1D(kernel_num, kernel_size, padding='same', activation='relu',
                                  dilation_rate=dialation)(bn1)
    bn2 = layers.BatchNormalization()(conv1d_layer1)
    conv1d_layer2 = layers.Conv1D(kernel_num, kernel_size, padding='same', activation='relu',
                                  dilation_rate=dialation)(bn2)
    return layers.Add()([input_layer, conv1d_layer2])


def resnet_1(input_layer, block_num=RESNET_1_BLOCKS, kernel_size=RESNET_1_KERNEL_SIZE,
             kernel_num=RESNET_1_KERNEL_NUM):
    """
    ResNet layer - input -> BatchNormalization -> Conv1D -> Relu -> BatchNormalization -> Conv1D -> Relu -> Add
    :param input_layer: input layer for the ResNet
    :return: last layer of the ResNet
    """
    last_layer_output = input_layer

    for i in range(block_num):
        last_layer_output = resnet_block(last_layer_output, kernel_size, kernel_num)

    return last_layer_output


def resnet_2(input_layer, block_num=RESNET_2_BLOCKS, kernel_size=RESNET_2_KERNEL_SIZE,
             kernel_num=RESNET_2_KERNEL_NUM, dial_lst=DILATION):
    """
    Dilated ResNet layer - input -> BatchNormalization -> dilated Conv1D -> Relu -> BatchNormalization -> dilated Conv1D -> Relu -> Add
    :param input_layer: input layer for the ResNet
    :return: last layer of the ResNet
    """
    last_layer_output = input_layer

    for i in range(block_num):
        for d in dial_lst:
            last_layer_output = resnet_block(last_layer_output, kernel_size, kernel_num, d)

    return last_layer_output

def get_default_config():
    sweep_config = {'RESNET_1_BLOCKS': RESNET_1_BLOCKS,
                    'RESNET_1_KERNEL_SIZE': RESNET_1_KERNEL_SIZE,
                    'RESNET_1_KERNEL_NUM': RESNET_1_KERNEL_NUM,
                    'RESNET_2_BLOCKS': RESNET_2_BLOCKS,
                    'RESNET_2_KERNEL_SIZE': RESNET_2_KERNEL_SIZE,
                    'RESNET_2_KERNEL_NUM': RESNET_2_KERNEL_NUM,
                    'DROPOUT': DROPOUT, 'EPOCHS': EPOCHS, "LR": LR,
                    'DILATATION': DILATION, 'BATCH': BATCH, 'method': 'random',
                    'metric': {'name': 'loss', 'goal': 'minimize'},
                    'name': f"BioEx4_{get_time()}"}

    return sweep_config

def build_network(config=None):
    """
    builds the neural network architecture as shown in the exercise.
    :return: a Keras Model
    """
    if config is None:
        config = get_default_config()

    # input, shape (NB_MAX_LENGTH,FEATURE_NUM)
    input_layer = tf.keras.Input(shape=(utils.NB_MAX_LENGTH, utils.FEATURE_NUM))

    # Conv1D -> shape = (NB_MAX_LENGTH, RESNET_1_KERNEL_NUM)
    conv1d_layer = layers.Conv1D(config['RESNET_1_KERNEL_NUM'], config['RESNET_1_KERNEL_SIZE'],
                                 padding='same')(input_layer)

    # first ResNet -> shape = (NB_MAX_LENGTH, RESNET_1_KERNEL_NUM)
    resnet_layer = resnet_1(conv1d_layer, config['RESNET_1_BLOCKS'], config['RESNET_1_KERNEL_SIZE'],
                            config['RESNET_1_KERNEL_NUM'])

    # Conv1D -> shape = (NB_MAX_LENGTH, RESNET_2_KERNEL_NUM)
    conv1d_layer = layers.Conv1D(config['RESNET_2_KERNEL_NUM'], config['RESNET_2_KERNEL_SIZE'],
                                 padding="same")(resnet_layer)

    # second ResNet -> shape = (NB_MAX_LENGTH, RESNET_2_KERNEL_NUM)
    resnet_layer = resnet_2(conv1d_layer, config['RESNET_2_BLOCKS'], config['RESNET_2_KERNEL_SIZE'],
                            config['RESNET_2_KERNEL_NUM'], config['DILATATION'])

    dp = layers.Dropout(config['DROPOUT'])(resnet_layer)
    conv1d_layer = layers.Conv1D(config['RESNET_2_KERNEL_NUM'] // 2, config['RESNET_2_KERNEL_SIZE'],
                                 padding="same",
                                 activation='elu')(dp)
    dense = layers.Dense(15)(conv1d_layer)

    return tf.keras.Model(input_layer, dense)


def plot_val_train_loss(history):
    """
    plots the train and validation loss of the model at each epoch, saves it in 'model_loss_history.png'
    :param history: history object (output of fit function)
    :return: None
    """
    ig, axes = plt.subplots(1, 1, figsize=(15, 3))
    axes.plot(history.history['loss'], label='Training loss')
    axes.plot(history.history['val_loss'], label='Validation loss')
    axes.legend()
    axes.set_title("Train and Val MSE loss")

    plt.savefig("/content/drive/MyDrive/Ex4Files/model_loss_history")  # TODO: you can change the path here


def get_config():
    sweep_config = {}
    sweep_config['method'] = 'bayes'
    sweep_config['metric'] = {'name': 'best_val_loss', 'goal': 'minimize'}
    sweep_config["early_terminate"]= {
        "type": "hyperband",
        "min_iter": 2,
        "eta": 2,
    }

    sweep_config['name'] = f"BioEx4_{get_time()}"
    param_dict = {
        'RESNET_1_BLOCKS': {'distribution': 'int_uniform', 'min': 1, 'max': 5},
        'RESNET_1_KERNEL_SIZE': {'values': [3, 5, 7, 9]},
        'RESNET_1_KERNEL_NUM': {'distribution': 'int_uniform', 'min': 8,
                                'max': 64},
        'RESNET_2_BLOCKS': {'distribution': 'int_uniform', 'min': 1, 'max': 5},
        'RESNET_2_KERNEL_SIZE': {'values': [3, 5, 7, 9]},
        'RESNET_2_KERNEL_NUM': {'distribution': 'int_uniform', 'min': 8,
                                'max': 64},
        'DROPOUT': {'distribution': 'uniform', 'min': 0.001, 'max': 0.5},
        'EPOCHS': {'distribution': 'int_uniform', 'min': 5, 'max': 15},
        "LR": {'distribution': 'uniform', 'min': 0.001, 'max': 0.025},
        'BATCH': {'values': [16, 32, 64, 128, 256]},
        'DILATATION': {'values': [[1, 2, 4], [1], [1, 2], [1, 4], [1, 2, 4, 8]]}
    }

    sweep_config['parameters'] = param_dict
    return sweep_config





class WandbCallback(tf.keras.callbacks.Callback):
    def __init__(self, fold):
        super(WandbCallback, self).__init__()
        self.fold = fold

    def on_epoch_end(self, epoch, logs=None):
        wandb.log({'loss': logs['loss'], 'val_loss': logs['val_loss'], 'fold':
            self.fold, 'epoch': epoch})


def train(config=None):
    if config is None:
        config = get_default_config()
    with wandb.init(config=config) as run:

        # _______________loading the data_______________
        config = wandb.config
        input = np.load("train_input.npy")  # numpy array of shape (1974,NB_MAX_LENGTH,FEATURE_NUM) - data
        labels = np.load("train_labels.npy")  # numpy array of shape (1974,NB_MAX_LENGTH,OUTPUT_SIZE) - labels
        save_dir = "BestFits/"
        model_name = run.name
        fold_var = 1
        kf = KFold(n_splits=5, shuffle=True, random_state=0)
        my_optimizer = tf.keras.optimizers.Adam(learning_rate=config['LR'])
        loss = 0
        losses = np.zeros(5)
        for t_idx, v_idx in kf.split(input, labels):
            X_t, X_v = input[t_idx], input[v_idx]
            y_t, y_v = labels[t_idx], labels[v_idx]

            model = build_network(config)
            # _______________compiling______________

            model.compile(optimizer=my_optimizer, loss='mean_squared_error')

            # _____________creating callbacks_____________
            checkpoint = tf.keras.callbacks.ModelCheckpoint(f"{save_dir}"
                                                            f"{model_name}"
                                                            f"{fold_var}.ckpt",
                                                            monitor='val_loss',
                                                            save_best_only=True, mode='min')

            callbacks_list = [checkpoint, WandbCallback(fold_var)]

            # _____________fitting the model______________
            history = model.fit(X_t, y_t,
                                epochs=config['EPOCHS'],
                                callbacks=callbacks_list,
                                batch_size=config['BATCH'],
                                validation_data=(X_v, y_v))


            # _____________evaluate the model_____________
            best_model = tf.keras.models.load_model(f"{save_dir}"
                                                    f"{model_name}"
                                                    f"{fold_var}.ckpt")

            l = best_model.evaluate(X_v, y_v)
            losses[fold_var - 1] = l
            loss += l/5
            wandb.log({'best_val_loss': loss})
            # loss[fold_var - 1] = best_model.evaluate(X_v, y_v)
            fold_var += 1
            tf.keras.backend.clear_session()
        wandb.log({'mean_loss': loss,'std':np.std(losses)})



In [ ]:
def main():
    sweep_id = wandb.sweep(get_config(), project="BioEx4_5",
                           entity="avishai-elma")
    wandb.agent(sweep_id, train, count=1000)


In [ ]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:

main()
    



Create sweep with ID: 5pa1928v
Sweep URL: https://wandb.ai/avishai-elma/BioEx4_5/sweeps/5pa1928v


wandb: Agent Starting Run: mky53j82 with config:
wandb: 	BATCH: 16
wandb: 	DILATATION: [1, 2, 4, 8]
wandb: 	DROPOUT: 0.24813238652338945
wandb: 	EPOCHS: 10
wandb: 	LR: 0.02401794717399047
wandb: 	RESNET_1_BLOCKS: 3
wandb: 	RESNET_1_KERNEL_NUM: 18
wandb: 	RESNET_1_KERNEL_SIZE: 9
wandb: 	RESNET_2_BLOCKS: 4
wandb: 	RESNET_2_KERNEL_NUM: 8
wandb: 	RESNET_2_KERNEL_SIZE: 3
wandb: Currently logged in as: avishai-elma. Use `wandb login --relogin` to force relogin


Epoch 1/10
99/99 [==============================] - 58s 333ms/step - loss: 29.3807 - val_loss: 25.9727
Epoch 2/10
99/99 [==============================] - 31s 312ms/step - loss: 10.6684 - val_loss: 10.2870
Epoch 3/10
99/99 [==============================] - ETA: 0s - loss: 7.5076

wandb: ERROR Error while calling W&B API: context deadline exceeded (<Response [500]>)


INFO:tensorflow:Assets written to: BestFits/lunar-sweep-11.ckpt/assets
99/99 [==============================] - 30s 309ms/step - loss: 7.5076 - val_loss: 7.5742
Epoch 4/10
99/99 [==============================] - 31s 318ms/step - loss: 6.1204 - val_loss: 4.2648
Epoch 5/10
92/99 [==========================>...] - ETA: 0s - loss: 5.4696

...

wandb: Agent Starting Run: ec48db2f with config:
wandb: 	BATCH: 16
wandb: 	DILATATION: [1, 2, 4, 8]
wandb: 	DROPOUT: 0.13131639632557535
wandb: 	EPOCHS: 14
wandb: 	LR: 0.0150084979997621
wandb: 	RESNET_1_BLOCKS: 4
wandb: 	RESNET_1_KERNEL_NUM: 59
wandb: 	RESNET_1_KERNEL_SIZE: 5
wandb: 	RESNET_2_BLOCKS: 5
wandb: 	RESNET_2_KERNEL_NUM: 9
wandb: 	RESNET_2_KERNEL_SIZE: 7


Epoch 1/14
99/99 [==============================] - 57s 387ms/step - loss: 53.1028 - val_loss: 52.6792
Epoch 2/14
99/99 [==============================] - 36s 366ms/step - loss: 53.0642 - val_loss: 52.6773
Epoch 3/14
99/99 [==============================] - 9s 94ms/step - loss: 53.0644 - val_loss: 52.6784
Epoch 4/14
99/99 [==============================] - 9s 93ms/step - loss: 53.0632 - val_loss: 52.6909
Epoch 5/14
99/99 [==============================] - 9s 92ms/step - loss: 53.0642 - val_loss: 52.6781
Epoch 6/14
99/99 [==============================] - 9s 93ms/step - loss: 53.0638 - val_loss: 52.6777
Epoch 7/14
99/99 [==============================] - 36s 365ms/step - loss: 53.0625 - val_loss: 52.6765
Epoch 8/14
99/99 [==============================] - 9s 91ms/step - loss: 53.0636 - val_loss: 52.6813
Epoch 9/14
99/99 [==============================] - 9s 93ms/step - loss: 53.0654 - val_loss: 52.6798
Epoch 10/14
99/99 [==============================] - 9s 92ms/step - loss: 53.0644 - v

best_val_loss,▁▃▄▆█
epoch,▁▂▃▄▅▅▆▇▁▂▃▄▅▆▆▇▁▂▃▄▅▆▆▇▁▂▃▄▅▆▇▇▁▂▃▄▅▆▇█
fold,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████
loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mean_loss,▁
std,▁
val_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁
best_val_loss,52.98457
epoch,13
fold,5
loss,52.89019


wandb: Agent Starting Run: ovi1xo4p with config:
wandb: 	BATCH: 32
wandb: 	DILATATION: [1, 2, 4, 8]
wandb: 	DROPOUT: 0.4738042427355925
wandb: 	EPOCHS: 14
wandb: 	LR: 0.00424400901080143
wandb: 	RESNET_1_BLOCKS: 3
wandb: 	RESNET_1_KERNEL_NUM: 61
wandb: 	RESNET_1_KERNEL_SIZE: 9
wandb: 	RESNET_2_BLOCKS: 5
wandb: 	RESNET_2_KERNEL_NUM: 12
wandb: 	RESNET_2_KERNEL_SIZE: 3


Epoch 1/14
50/50 [==============================] - 51s 706ms/step - loss: 39.7595 - val_loss: 36.3837
Epoch 2/14
50/50 [==============================] - 31s 621ms/step - loss: 17.9329 - val_loss: 25.6791
Epoch 3/14
50/50 [==============================] - 5s 102ms/step - loss: 9.4779 - val_loss: 37.7454
Epoch 4/14
50/50 [==============================] - 31s 629ms/step - loss: 7.2485 - val_loss: 11.4262
Epoch 5/14
50/50 [==============================] - 31s 636ms/step - loss: 6.7011 - val_loss: 7.4347
Epoch 6/14
50/50 [==============================] - 30s 620ms/step - loss: 6.3117 - val_loss: 5.7732
Epoch 7/14
50/50 [==============================] - 31s 637ms/step - loss: 6.0929 - val_loss: 3.5523
Epoch 8/14
50/50 [==============================] - 5s 103ms/step - loss: 5.9284 - val_loss: 3.6723
Epoch 9/14
50/50 [==============================] - 31s 623ms/step - loss: 5.7977 - val_loss: 2.8412
Epoch 10/14
50/50 [==============================] - 30s 616ms/step - loss: 5.6587 - va